In [1]:
import requests
import pandas as pd 
import json
import numpy as np
from datetime import datetime

In [2]:
# endpoint = "https://wikimedia.org/api/rest_v1"

In [3]:
# def return_url(article_name):
#     """ Returns url for specified name
#     """
#     project = "en.wikipedia.org"
#     access = "all-access"
#     agent = "all-agents"
#     article = article_name
#     granularity = "monthly"
#     start = "20010101"
#     end = "20170101"
#     params = "/metrics/pageviews/per-article/{}/{}/{}/{}/{}/{}/{}".format(project,access,
#     agent, article, granularity, start, end)
#     return params

In [4]:
# headers = {'content-type': 'application/json'}
# r = requests.get(endpoint+return_params("Adolf_Hitler"), headers = headers)

In [5]:
# print(r.content.decode())
# resp_dict = json.loads(r.content.decode())

In [2]:
def get_views(name): 
    """ Returns dictionary with monthly view numbers
    """
    headers = {'content-type': 'application/json'}
    r = requests.get(endpoint+return_url(name), headers = headers)
    resp_dict = json.loads(r.content.decode())
    return resp_dict

In [3]:
def month_views(resp_dict, month, year):
    time = str(year)+str(month)+"0100"
    if 'items' not in resp_dict.keys():
#         print(resp_dict)
        return 0
    for dic in resp_dict['items']:
#         print(dic['timestamp'])
#         print("{} - {}".format(dic['timestamp'], time))
        if dic['timestamp'] == time:
            return dic['views']
    return 0 

In [4]:
def cum_views(resp_dict, year):
    """ Retruns cumulative number of views for page in specified year
    """
    if 'items' not in resp_dict.keys():
#         print(resp_dict)
        return 0
    views = 0
    for dic in resp_dict['items']:
        if dic['timestamp'][:4] == str(year):
            views += dic['views']
    return views

In [5]:
data = pd.read_json("../data/politicians_with_gender.json")
# wiki_urls = pd.read_json("../data/politicians_with_gender.json").WikiURL.values
# wiki_urls[1].split("/wiki/")[1]

In [6]:
data["url_name"] = data["WikiURL"].apply(lambda x: x.split("/wiki/")[1])

In [7]:
names_df = data[["ID", "url_name"]]
len(names_df)

69179

In [12]:
# article_names = [url.split("/wiki/")[1] for url in wiki_urls]

In [13]:
# names_df["url_name"].values

In [8]:
df1, df2, df3, df4 = np.array_split(names_df, 4)

In [15]:
# len(df4)

In [16]:
# df1.to_csv("set1.csv", index=False, encoding="utf-8")
# df2.to_csv("set2.csv", index=False, encoding="utf-8")
# df3.to_csv("set3.csv", index=False, encoding="utf-8")
# df4.to_csv("set4.csv", index=False, encoding="utf-8")

In [17]:
# path = "set1.csv"

# names = pd.read_csv(path)["url_name"].values

In [18]:
# count = 1
# lst = []
# for name in names[:100]:
#     dic = get_views(name)
# #     print(cum_views(dic, 2016))
#     sublst = [name, dic]
#     lst.append(sublst)
#     print("{} - {} - {}".format(count ,datetime.now(), name))
#     count = count + 1

In [19]:
# get_views('Barack_Obama')

In [20]:
# pd.DataFrame(lst)

In [9]:
set1 = pd.read_json("set1.json")

In [10]:
set2 = pd.read_json("set2.json")

In [11]:
set3 = pd.read_json("set3.json")

In [12]:
set4 = pd.read_json("set4.json")

In [13]:
page_views = pd.concat([set1, set2, set3, set4])

In [14]:
page_views.columns = ["url_name", "view_dict"]

In [15]:
page_views["views"] = page_views["view_dict"].apply(lambda x: cum_views(x, 2016))

In [16]:
page_views["dec2016"] = page_views['view_dict'].apply(lambda x: month_views(x, 12, 2016))

In [17]:
page_views.head()

,url_name,view_dict,views,dec2016
0,Quincy_Timberlake,"{'items': [{'project': 'en.wikipedia', 'articl...",5106,262
1,Fanny_Edelman,"{'items': [{'project': 'en.wikipedia', 'articl...",5228,564
10,Luis_Cabrera_Lobato,"{'items': [{'project': 'en.wikipedia', 'articl...",4492,334
100,Aishat_Ismail,"{'items': [{'project': 'en.wikipedia', 'articl...",1755,147
1000,Henry_Tang,"{'items': [{'project': 'en.wikipedia', 'articl...",19589,2264


In [18]:
# page_views = page_views.reset_index()
page_views.to_pickle("page_views")

In [22]:
result = pd.merge(data, page_views, on=['url_name', 'url_name'])
result = result.drop(["view_dict"], axis=1)
result.head()

,#DBpURL,ID,WikiURL,birthDate,deathDate,first_name,fpp_gender,full_name,gender,genderize,name,nationality,occupation,party,url_name,views,dec2016
0,http://dbpedia.org/resource/Quincy_Timberlake,11,http://en.wikipedia.org/wiki/Quincy_Timberlake,3.252096e+11,NaN,quincy,None,Quincy+Timberlake,male,None,"[ quincy timberlake , timberlake quincy ]",[kenyan],[politician],[],Quincy_Timberlake,5106,262
1,http://dbpedia.org/resource/Fanny_Edelman,29,http://en.wikipedia.org/wiki/Fanny_Edelman,-1.856995e+12,1.320106e+12,fanny,None,Fanny+Edelman,female,None,"[ fanny edelman , edelman fanny ]",[argentine],[politician],[],Fanny_Edelman,5228,564
2,http://dbpedia.org/resource/Georges_Colombier,252,http://en.wikipedia.org/wiki/Georges_Colombier,-9.409824e+11,NaN,georges,None,Georges+Colombier,male,None,[ colombier georges ],[french],[politician],[ union for a popular movement ],Georges_Colombier,841,94
3,http://dbpedia.org/resource/Mira_Spivak,1858,http://en.wikipedia.org/wiki/Mira_Spivak,-1.119485e+12,NaN,mira,None,Mira+Spivak,female,None,[ mira spivak ],"[canadian, ukrainian]",[politician],[ progressive conservative party of canada ],Mira_Spivak,1762,180
4,http://dbpedia.org/resource/Auguste_Beernaert,16847,http://en.wikipedia.org/wiki/Auguste_Beernaert,-4.431715e+12,-1.806278e+12,auguste,None,Auguste+Beernaert,male,male,[ auguste beernaert ],[belgian],[politician],[ catholic party (belgium) ],Auguste_Beernaert,14868,1229


In [ ]:
# result.drop(["view_dict"],1)
# result.to_json("page_views.json")

In [19]:
# result.view_dict.to_json("page_views_dict.json")
# result.drop(["view_dict"],1)
# result.to_json("page_views.json")

In [20]:
# with open('page_views_dict.json', 'w') as fp:
#     json.dump(dic, fp)

In [23]:
result.to_json("politicians_pw.json")